In [1]:
import numpy as np
import string

from qiskit.circuit.library import TwoLocal
from qiskit import QuantumCircuit
from qiskit.transpiler.preset_passmanagers import generate_preset_pass_manager
from qiskit.quantum_info import SparsePauliOp

from qiskit_addon_cutting import partition_problem
from qiskit_addon_cutting import generate_cutting_experiments

from qiskit_aer import AerSimulator
from qiskit_ibm_runtime import SamplerV2, Batch

In [4]:
# constructing quantum circuit of same structure as given in the qasm file

num_qubits = 4
reps = 4
gate_sequence = ['rz','sx','rz','sx','x','rz']

# nearest neighbour configuration
seq_pairs = [(i,(i+1)%num_qubits) for i in range(0,num_qubits,2)]
warp_pairs = [(i,(i-1)%num_qubits) for i in range(0,num_qubits,2)]

# qc two local
qc = TwoLocal(num_qubits,rotation_blocks=['rz','sx','rz','sx','x','rz'],entanglement_blocks='cz', entanglement=[seq_pairs,warp_pairs], reps=reps,skip_final_rotation_layer=True).decompose()  
qc.assign_parameters([np.random.random(1)[0] for i in range(len(qc.parameters))], inplace=True)
#qc.measure_all()
qc.draw(scale=0.8)

┌─────────────┐ ┌────┐┌─────────────┐┌────┐┌───┐ ┌────────────┐    »
q_0: ┤ Rz(0.22697) ├─┤ √X ├┤ Rz(0.45817) ├┤ √X ├┤ X ├─┤ Rz(0.3613) ├──■─»
     ├─────────────┤ ├────┤├─────────────┤├────┤├───┤┌┴────────────┤  │ »
q_1: ┤ Rz(0.70913) ├─┤ √X ├┤ Rz(0.62439) ├┤ √X ├┤ X ├┤ Rz(0.67894) ├──■─»
     ├─────────────┴┐├────┤├─────────────┤├────┤├───┤├─────────────┤    »
q_2: ┤ Rz(0.080658) ├┤ √X ├┤ Rz(0.17078) ├┤ √X ├┤ X ├┤ Rz(0.28971) ├──■─»
     ├──────────────┤├────┤├─────────────┤├────┤├───┤├─────────────┴┐ │ »
q_3: ┤ Rz(0.034409) ├┤ √X ├┤ Rz(0.73363) ├┤ √X ├┤ X ├┤ Rz(0.068062) ├─■─»
     └──────────────┘└────┘└─────────────┘└────┘└───┘└──────────────┘   »
«     ┌──────────────┐┌────┐┌─────────────┐┌────┐┌───┐┌─────────────┐       »
«q_0: ┤ Rz(0.018794) ├┤ √X ├┤ Rz(0.41935) ├┤ √X ├┤ X ├┤ Rz(0.24384) ├─────■─»
«     ├─────────────┬┘├────┤├─────────────┤├────┤├───┤├─────────────┤     │ »
«q_1: ┤ Rz(0.72749) ├─┤ √X ├┤ Rz(0.95011) ├┤ √X ├┤ X ├┤ Rz(0.57923) ├──■──┼─»
«     ├─────────────┤ ├────┤├─────────────┤├────┤├───┤├─────────────┴┐ │  │ »
«q_2: ┤ Rz(0.16075) ├─┤ √X ├┤ Rz(0.93514) ├┤ √X ├┤ X ├┤ Rz(0.090199) ├─■──┼─»
«     ├─────────────┤ ├────┤├─────────────┤├────┤├───┤├─────────────┬┘    │ »
«q_3: ┤ Rz(0.60459) ├─┤ √X ├┤ Rz(0.57327) ├┤ √X ├┤ X ├┤ Rz(0.11734) ├─────■─»
«     └─────────────┘ └────┘└─────────────┘└────┘└───┘└─────────────┘       »
«     ┌─────────────┐┌────┐┌─────────────┐┌────┐┌───┐ ┌────────────┐   »
«q_0: ┤ Rz(0.98626) ├┤ √X ├┤ Rz(0.84015) ├┤ √X ├┤ X ├─┤ Rz(0.7159) ├─■─»
«     ├─────────────┤├────┤├─────────────┤├────┤├───┤┌┴────────────┤ │ »
«q_1: ┤ Rz(0.49289) ├┤ √X ├┤ Rz(0.37398) ├┤ √X ├┤ X ├┤ Rz(0.74452) ├─■─»
«     ├─────────────┤├────┤├─────────────┤├────┤├───┤├─────────────┤   »
«q_2: ┤ Rz(0.91152) ├┤ √X ├┤ Rz(0.41194) ├┤ √X ├┤ X ├┤ Rz(0.74607) ├─■─»
«     ├─────────────┤├────┤├─────────────┤├────┤├───┤├─────────────┤ │ »
«q_3: ┤ Rz(0.76347) ├┤ √X ├┤ Rz(0.68385) ├┤ √X ├┤ X ├┤ Rz(0.49188) ├─■─»
«     └─────────────┘└────┘└─────────────┘└────┘└───┘└─────────────┘   »
«      ┌─────────────┐ ┌────┐┌─────────────┐┌────┐┌───┐┌─────────────┐      
«q_0: ─┤ Rz(0.12857) ├─┤ √X ├┤ Rz(0.43553) ├┤ √X ├┤ X ├┤ Rz(0.42043) ├────■─
«      ├─────────────┤ ├────┤├─────────────┤├────┤├───┤├─────────────┤    │ 
«q_1: ─┤ Rz(0.95693) ├─┤ √X ├┤ Rz(0.20394) ├┤ √X ├┤ X ├┤ Rz(0.38673) ├─■──┼─
«      ├─────────────┤ ├────┤├─────────────┤├────┤├───┤├─────────────┤ │  │ 
«q_2: ─┤ Rz(0.24545) ├─┤ √X ├┤ Rz(0.26362) ├┤ √X ├┤ X ├┤ Rz(0.82642) ├─■──┼─
«     ┌┴─────────────┴┐├────┤├─────────────┤├────┤├───┤├─────────────┤    │ 
«q_3: ┤ Rz(0.0086867) ├┤ √X ├┤ Rz(0.78625) ├┤ √X ├┤ X ├┤ Rz(0.56725) ├────■─
«     └───────────────┘└────┘└─────────────┘└────┘└───┘└─────────────┘

### Parallelising